In [2]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2

In [3]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--images", required=True, help="path to images directory")

_StoreAction(option_strings=['-i', '--images'], dest='images', nargs=None, const=None, default=None, type=None, choices=None, help='path to images directory', metavar=None)

In [4]:
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGES
ipykernel_launcher.py: error: argument -i/--images is required


SystemExit: 2

/Users/tlenglin/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# initialize a rectangular and square structuring kernel
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 30))

In [6]:
# loop over the input image paths
for imagePath in paths.list_images("./images"):
    # load the image, resize it, and convert it to grayscale
    image = cv2.imread(imagePath)
    image = imutils.resize(image, height=600)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    # smooth the image using a 3x3 Gaussian, then apply the blackhat
    # morphological operator to find dark regions on a light background
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)

In [7]:
blackhat

array([[0, 0, 0, ..., 1, 2, 2],
       [0, 0, 0, ..., 2, 2, 2],
       [0, 0, 0, ..., 2, 2, 2],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=uint8)

In [8]:
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")

In [9]:
# apply a closing operation using the rectangular kernel to close
# gaps in between letters -- then apply Otsu's thresholding method
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)


In [10]:
cv2.imshow("TEST-1", gradX)
cv2.waitKey(0)

0

In [11]:
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [12]:
cv2.imshow("TEST0", thresh)
cv2.waitKey(0)

0

In [13]:
# perform another closing operation, this time using the square
# kernel to close gaps between lines of the MRZ, then perform a
# series of erosions to break apart connected components
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
thresh = cv2.erode(thresh, None, iterations=4)

In [14]:
cv2.imshow("TEST", thresh)
cv2.waitKey(0)

0

In [15]:
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, rectKernel)

In [16]:
cv2.imshow("TEST2", thresh)
cv2.waitKey(0)

0

In [17]:
# during thresholding, it's possible that border pixels were
# included in the thresholding, so let's set 5% of the left and
# right borders to zero
p = int(image.shape[1] * 0.05)
thresh[:, 0:p] = 0
thresh[:, image.shape[1] - p:] = 0

In [18]:
# find contours in the thresholded image and sort them by their
# size
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)[-2]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [19]:
# loop over the contours
for c in cnts:
    # compute the bounding box of the contour and use the contour to
    # compute the aspect ratio and coverage ratio of the bounding box
    # width to the width of the image
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    crWidth = w / float(gray.shape[1])
 
    # check to see if the aspect ratio and coverage width are within
    # acceptable criteria
    if ar > 5 and crWidth > 0.75:
        # pad the bounding box since we applied erosions and now need
        # to re-grow it
        pX = int((x + w) * 0.03)
        pY = int((y + h) * 0.03)
        (x, y) = (x - pX, y - pY)
        (w, h) = (w + (pX * 2), h + (pY * 2))
 
        # extract the ROI from the image and draw a bounding box
        # surrounding the MRZ
        roi = image[y:y + h, x:x + w].copy()
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        break

In [ ]:
# show the output images
cv2.imshow("Image", image)
cv2.imshow("ROI", roi)
cv2.waitKey(0)

array([[[193,  99,   0],
        [194, 100,   0],
        [194, 100,   0],
        ..., 
        [204, 103,   1],
        [206, 105,   2],
        [208, 107,   5]],

       [[193,  99,   0],
        [194, 100,   1],
        [194, 100,   1],
        ..., 
        [206, 105,   2],
        [208, 107,   5],
        [210, 109,   7]],

       [[198, 104,   4],
        [197, 103,   3],
        [196, 102,   2],
        ..., 
        [207, 106,   3],
        [209, 108,   5],
        [209, 108,   6]],

       ..., 
       [[200, 121,  63],
        [208, 130,  72],
        [206, 129,  71],
        ..., 
        [166, 101,  13],
        [163,  98,   9],
        [157,  92,   3]],

       [[200, 124,  67],
        [207, 130,  74],
        [201, 126,  69],
        ..., 
        [167, 101,  12],
        [164,  98,   8],
        [157,  91,   1]],

       [[194, 119,  63],
        [196, 122,  66],
        [193, 121,  63],
        ..., 
        [165,  98,   8],
        [164,  99,   7],
        [162,  97,